<a href="https://colab.research.google.com/github/niharikasingh3632/Mental-Health-Counseling-Summarization/blob/main/Processing_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [12]:
!pip install spacy inflect
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 49.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [18]:
import os
import pandas as pd
import spacy
import inflect

# Load NLP tools
nlp = spacy.load("en_core_web_sm")
inflect_engine = inflect.engine()

def adjust_utterance(text, role):
    doc = nlp(text)
    new_tokens = []
    skip_next = False

    for i, token in enumerate(doc):
        if skip_next:
            skip_next = False
            continue

        if token.text.lower() in ["i'm"]:
            new_tokens.append(role)
            continue

        if token.text.lower() == "i":
            new_tokens.append(role)
            if i + 1 < len(doc):
                next_token = doc[i + 1]
                if next_token.tag_ == "VBP":
                    corrected = inflect_engine.plural_verb(next_token.text)
                    new_tokens.append(corrected)
                    skip_next = True
            continue

        new_tokens.append(token.text)

    return " ".join(new_tokens)

def preprocess_csv_files(input_path, output_path, train_file_path):
    os.makedirs(output_path, exist_ok=True)
    processed_file_names = []

    for filename in os.listdir(input_path):
        if not filename.endswith(".csv"):
            continue

        file_path = os.path.join(input_path, filename)
        try:
            df = pd.read_csv(file_path, encoding="utf-8")
            df.columns = df.columns.str.strip()

            if len(df) >= 3:
                metadata_df = df.iloc[-3:].copy()
                dialogue_df = df.iloc[:-3].copy()
            else:
                metadata_df = pd.DataFrame()
                dialogue_df = df.copy()

            if "Sub topic" in dialogue_df.columns:
                original_len = len(dialogue_df)
                dialogue_df = dialogue_df[~dialogue_df["Sub topic"].str.lower().eq("inactive")]
                removed = original_len - len(dialogue_df)
                print(f"{removed} 'inactive' rows removed in {filename}")

            if dialogue_df.empty:
                print(f"⚠️ No valid dialogue rows to process in: {filename}")
                continue

            if "Utterance" not in dialogue_df.columns or "Type" not in dialogue_df.columns:
                raise KeyError(f"Required columns missing in file: {filename}")

            def process_row(row):
                utterance = str(row["Utterance"])
                role = "Patient" if row["Type"] == "P" else "Therapist"
                return adjust_utterance(utterance, role)

            dialogue_df["Utterance"] = dialogue_df.apply(process_row, axis=1)

            final_df = pd.concat([dialogue_df, metadata_df], ignore_index=True)

            output_file = os.path.join(output_path, filename)
            final_df.to_csv(output_file, index=False)

            if os.path.exists(output_file):
                print(f"Saved: {output_file}")
                processed_file_names.append(os.path.splitext(filename)[0])
            else:
                print(f"Failed to save file: {output_file}")

        except Exception as e:
            print(f"Error processing {filename}: {e}")

    # Write processed filenames
    with open(train_file_path, "w") as f:
        for name in processed_file_names:
            f.write(name + "\n")

    print(f"\n Done! {len(processed_file_names)} files saved to: {output_path}")
    print(f" Log saved to: {train_file_path}")


In [19]:
input_path = "/content/drive/MyDrive/MEMO_KDD_2022/Train"
output_path = "/content/drive/MyDrive/MEMO_KDD_2022/Processed_new/training"
train = "/content/drive/MyDrive/MEMO_KDD_2022/Processed_new/train.txt"

preprocess_csv_files(input_path, output_path, train)


0 'inactive' rows removed in 34.csv
Saved: /content/drive/MyDrive/MEMO_KDD_2022/Processed_new/training/34.csv
1 'inactive' rows removed in 48.csv
Saved: /content/drive/MyDrive/MEMO_KDD_2022/Processed_new/training/48.csv
2 'inactive' rows removed in 66.csv
Saved: /content/drive/MyDrive/MEMO_KDD_2022/Processed_new/training/66.csv
5 'inactive' rows removed in 107.csv
Saved: /content/drive/MyDrive/MEMO_KDD_2022/Processed_new/training/107.csv
3 'inactive' rows removed in 109.csv
Saved: /content/drive/MyDrive/MEMO_KDD_2022/Processed_new/training/109.csv
10 'inactive' rows removed in 110.csv
Saved: /content/drive/MyDrive/MEMO_KDD_2022/Processed_new/training/110.csv
1 'inactive' rows removed in 111.csv
Saved: /content/drive/MyDrive/MEMO_KDD_2022/Processed_new/training/111.csv
0 'inactive' rows removed in 1.csv
Saved: /content/drive/MyDrive/MEMO_KDD_2022/Processed_new/training/1.csv
15 'inactive' rows removed in 3.csv
Saved: /content/drive/MyDrive/MEMO_KDD_2022/Processed_new/training/3.csv
12 '

In [20]:
input_path = "/content/drive/MyDrive/MEMO_KDD_2022/Validation"
output_path = "/content/drive/MyDrive/MEMO_KDD_2022/Processed_new/validating"
train = "/content/drive/MyDrive/MEMO_KDD_2022/Processed_new/val.txt"

preprocess_csv_files(input_path, output_path, train)


6 'inactive' rows removed in 16.csv
Saved: /content/drive/MyDrive/MEMO_KDD_2022/Processed_new/validating/16.csv
5 'inactive' rows removed in 19.csv
Saved: /content/drive/MyDrive/MEMO_KDD_2022/Processed_new/validating/19.csv
11 'inactive' rows removed in 40.csv
Saved: /content/drive/MyDrive/MEMO_KDD_2022/Processed_new/validating/40.csv
16 'inactive' rows removed in 46.csv
Saved: /content/drive/MyDrive/MEMO_KDD_2022/Processed_new/validating/46.csv
33 'inactive' rows removed in 69.csv
Saved: /content/drive/MyDrive/MEMO_KDD_2022/Processed_new/validating/69.csv
4 'inactive' rows removed in 72.csv
Saved: /content/drive/MyDrive/MEMO_KDD_2022/Processed_new/validating/72.csv
4 'inactive' rows removed in 75.csv
Saved: /content/drive/MyDrive/MEMO_KDD_2022/Processed_new/validating/75.csv
1 'inactive' rows removed in 81.csv
Saved: /content/drive/MyDrive/MEMO_KDD_2022/Processed_new/validating/81.csv
3 'inactive' rows removed in 102.csv
Saved: /content/drive/MyDrive/MEMO_KDD_2022/Processed_new/valida

In [17]:
input_path = "/content/drive/MyDrive/MEMO_KDD_2022/Test"
output_path = "/content/drive/MyDrive/MEMO_KDD_2022/Processed_new/testing"
train = "/content/drive/MyDrive/MEMO_KDD_2022/Processed_new/test.txt"

preprocess_csv_files(input_path, output_path, train)


Processed: 2.csv
Processed: 4.csv
Processed: 9.csv
Processed: 23.csv
Processed: 31.csv
Processed: 38.csv
Processed: 39.csv
Processed: 41.csv
Processed: 51.csv
Processed: 52.csv
Processed: 59.csv
Processed: 79.csv
Processed: 82.csv
Processed: 86.csv
Processed: 92.csv
Processed: 97.csv
Processed: 100.csv
Processed: 104.csv
Processed: 105.csv
Processed: 108.csv
Processed: 114.csv
Processed: 124.csv
Processed: 129.csv
Processed: 132.csv
Processed: 136.csv
Processed: 137.csv
Processed: 138.csv
Processed: 142.csv
Processed: 144.csv
Processed: 145.csv
Processed: 158.csv
Processed: 159.csv
Processed: 172.csv
Processed: 176.csv
Processed: 185.csv
Processed: 186.csv
Processed: 189.csv
Processed: 191.csv
Processed: 166.csv

All done! 39 files processed.
File names saved to: /content/drive/MyDrive/MEMO_KDD_2022/Processed_new/test.txt
